In [ ]:
import numpy as np 
import pandas as pd 
import os

import gc
import json
from PIL import Image
import geoplot
import geopandas as gpd
from tqdm import tqdm
from scipy.fft import fft, ifft
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf

Data parsing/aligning to timestamps
(data source: https://www.kaggle.com/c/indoor-location-navigation/data)
(helper functions: https://github.com/location-competition/indoor-location-competition-20)

In [ ]:
#after reviewing,implementation rotation to original coordinates may be 
#erroneous, please refer to library documentation:
#ahrs.readthedocs.io/ -- library used for kalman filtering, also has other useful algorithms

In [ ]:
data_root = "../input/indoor-location-navigation/train"
metadata_root = "../input/indoor-location-navigation/metadata"
state = "5d2709e003f801723c32d896"
floor = "F2"

def list_files(startpath):
    """Show directory structure recursive like tree command.
    refered from https://stackoverflow.com/questions/9727673/list-directory-tree-structure-in-python
    """
    file_list = []
    for root, dirs, files in os.walk(startpath):
        for f in files:
#             pathname_curr = startpath+ "/"+os.path.basename(root)+"/"+f
            pathname_curr = startpath+"/"+f
            
            file_list.append(pathname_curr)
    return file_list

path_building_curr = data_root + "/" + state + "/" + floor
# path_building_curr_test = data_root + "/" + state + "/" + floor_test

filelistcurr= list_files(path_building_curr)
# filelistcurr_test= list_files(path_building_curr_test)

# filelistcurr

In [ ]:
from pathlib import Path
acce_ = {}
gyro_ = {}
magn_ = {}
ahrs_ = {}
wifi_ = {}
ibeacon_ = {}
waypoint_ = {}
filename_biggest_wifi = {}
for data_filename in filelistcurr:
    acce = []
    gyro = []
    magn = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []
    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue
        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue
        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            dist = line_data[7]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi, dist]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce_[data_filename] = np.array(acce)
    gyro_[data_filename] = np.array(gyro)
    magn_[data_filename] = np.array(magn)
    ahrs_[data_filename] = np.array(ahrs)
    wifi_[data_filename] = np.array(wifi)
    ibeacon_[data_filename] = np.array(ibeacon)
    waypoint_[data_filename] = np.array(waypoint)
    filename_biggest_wifi[data_filename] = len(wifi)

In [ ]:
train_wifi_file, test_wifi_file = sorted(filename_biggest_wifi, key=filename_biggest_wifi.get, reverse=True)[:2]
# filename_biggest_wifi
train_wifi = wifi_[train_wifi_file]
test_wifi = wifi_[test_wifi_file]

In [ ]:
train_ibeacon = ibeacon_[train_wifi_file]

In [ ]:
wifi_df_verbose= pd.DataFrame(train_wifi, columns = ['timestamp', 'ssid', 'bssid', 'rssi', 'lastseen_ts'])
wifi_df_verbose_test= pd.DataFrame(test_wifi, columns = ['timestamp', 'ssid', 'bssid', 'rssi', 'lastseen_ts'])
wifi_df_verbose = wifi_df_verbose.drop(labels=['ssid','lastseen_ts'],axis=1)

ibeacon_df_verbose= pd.DataFrame(train_ibeacon, columns = ['timestamp', 'beaconId', 'rssi', 'distance'])


In [ ]:
train_file = train_wifi_file

acce = pd.DataFrame(acce_[train_file], columns=["timestamp", "acc_x", "acc_y", "acc_z"])
gyro = pd.DataFrame(gyro_[train_file], columns=["timestamp", "gyro_x", "gyro_y", "gyro_z"])
magn = pd.DataFrame(magn_[train_file], columns=["timestamp", "magn_x", "magn_y", "magn_z"])
ahrs = pd.DataFrame(ahrs_[train_file], columns=["timestamp", "ahrs_x", "ahrs_y", "ahrs_z"])
data_sens_df = (
    acce.merge(gyro, on="timestamp")
    .merge(magn, on="timestamp")
    .merge(ahrs, on="timestamp")
)
data_sens_df["timestamp"] = data_sens_df["timestamp"].astype(int)

test_file = test_wifi_file

acce_test = pd.DataFrame(acce_[test_file], columns=["timestamp", "acc_x", "acc_y", "acc_z"])
gyro_test = pd.DataFrame(gyro_[test_file], columns=["timestamp", "gyro_x", "gyro_y", "gyro_z"])
magn_test = pd.DataFrame(magn_[test_file], columns=["timestamp", "magn_x", "magn_y", "magn_z"])
ahrs_test = pd.DataFrame(ahrs_[test_file], columns=["timestamp", "ahrs_x", "ahrs_y", "ahrs_z"])
data_sens_df_test = (
    acce_test.merge(gyro_test, on="timestamp")
    .merge(magn_test, on="timestamp")
    .merge(ahrs_test, on="timestamp")
)
data_sens_df_test["timestamp"] = data_sens_df_test["timestamp"].astype(int)



In [ ]:
waypoint_df = pd.DataFrame(waypoint_[train_wifi_file],columns=["timestamp","x","y",])
waypoint_df["timestamp"] = waypoint_df["timestamp"].astype(int)
waypoint_df["x"] = waypoint_df["x"].astype(float)
waypoint_df["y"] = waypoint_df["y"].astype(float)

waypoint_df_test = pd.DataFrame(waypoint_[test_wifi_file],columns=["timestamp","x","y",])
waypoint_df_test["timestamp"] = waypoint_df_test["timestamp"].astype(int)
waypoint_df_test["x"] = waypoint_df_test["x"].astype(float)
waypoint_df_test["y"] = waypoint_df_test["y"].astype(float)


In [ ]:
def addtimestamps(data_df_,waypoint_df_):
    a_ = data_df_['timestamp'].values
    bh_ = waypoint_df_['timestamp'].values
    bl_ = waypoint_df_['timestamp'].values
    bl_ = np.insert(bl_, 0, 0, axis=0)
    last_, bl_ = bl_[-1], bl_[:-1]

    i_, j_ = np.where((a_[:, None] >= bl_) & (a_[:, None] <= bh_))

    df_pts_ = pd.DataFrame(
        np.column_stack([data_df_.values[i_], waypoint_df_[['x','y']].values[j_]]),
        columns=data_df_.columns.append(waypoint_df_.columns[1:3])
    )
    return df_pts_

In [ ]:
data_sens_df_merged = addtimestamps(data_sens_df, waypoint_df)#data_sens_df.merge(waypoint_df,how='outer',left_index=True,right_index=True)
sens_ground_truth = data_sens_df_merged[['x','y']]
data_sens_df_merged = data_sens_df_merged.drop(labels=['x','y'],axis=1)

data_sens_df_merged_test = addtimestamps(data_sens_df_test, waypoint_df_test)#data_sens_df_test.merge(waypoint_df_test,how='outer',left_index=True,right_index=True)
sens_ground_truth_test = data_sens_df_merged_test[['x','y']]
data_sens_df_merged_test = data_sens_df_merged_test.drop(labels=['x','y'],axis=1)


In [ ]:
xpos_initial = sens_ground_truth['x'][0]
ypos_initial = sens_ground_truth['y'][0]

In [ ]:
from ahrs.common.quaternion import QuaternionArray
gyr_arr = data_sens_df_merged[['gyro_x','gyro_y','gyro_z']].values
acce_arr = data_sens_df_merged[['acc_x','acc_y','acc_z']].values
magn_arr = data_sens_df_merged[['magn_x','magn_y','magn_z']].values

# ekf_estimator = EKF(gyr=gyr_arr, acc=acce_arr,mag=magn_arr, frequency=50.0,q0=qtn.Quaternion([1,0,0,0]))
ekf_estimator = EKF(gyr=gyr_arr, acc=acce_arr,mag=magn_arr, frequency=50.0)

#quaternion positions
quaternion_estimates = QuaternionArray(ekf_estimator.Q)
#convert from quaternion to euler angles
eu_angles = quaternion_estimates.to_angles()

from scipy.spatial.transform import Rotation as R

poslistx = []
poslisty = []
poslistz = []

x=xpos_initial
y=ypos_initial
z=0
counter = 0
for i in quaternion_estimates:
    posnew = qtn.Quaternion(i).rotate([xpos_initial,ypos_initial,0])
    x = posnew[0]
    y = posnew[1]
    z = posnew[2]
    poslistx.append(x)
    poslisty.append(y) 
    poslistz.append(z) 
poslistx = np.array(poslistx)
poslisty = np.array(poslisty)
poslistz = np.array(poslistz)